In [38]:
!pip install boto

In [39]:
!pip install opensearch-py

In [40]:
!pip install requests-aws4auth

In [41]:
!pip install awswrangler

In [42]:
!pip install jsonpath-ng

In [68]:
#Imports
import json
import pandas as pd
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
from datetime import datetime
from Cache import TweetCache

In [69]:
def os_connection():
    host = 'search-twitter-7hgtnq2vyvlq53ckdx2l7nel4y.aos.us-east-2.on.aws'
    region = 'us-east-2'

    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(region = region, service = service, session_token = credentials.token, refreshable_credentials = credentials)

    os = OpenSearch(
        hosts = [{'host': host, 'port': 443}],
        http_auth = awsauth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection)
    
    return os

In [45]:
os = os_connection()

In [70]:
def convert_lst_to_str(original_list):
    list_as_string = ','.join(map(str, original_list))
    return list_as_string

In [71]:
def convert_str_to_lst(list_as_string):
    if list_as_string:
        string_as_list = list(map(int, list_as_string.split(',')))
    else:
        string_as_list = []
    return string_as_list

In [72]:
def check_doc(field_to_check, value_to_check, index):
    
    os = os_connection()
    
    match_query = {
    "query": {
        "match": {
            field_to_check: value_to_check
        }}}
    
    response = None
    while not response:
        try:
            response = os.search(index=index, body=match_query)
        except:
            os = os_connection()
            continue

    documents_exist = response["hits"]["total"]["value"] > 0
    documents = response["hits"]["hits"]
    
    return documents, documents_exist

In [73]:
def str_to_datetime(date_string):

    # Input string
    date_string = 'Wed Mar 25 14:17:28 +0000 2020'

    # Define the format of the input string
    date_format = '%a %b %d %H:%M:%S %z %Y'

    # Convert the string to a datetime object
    date_time_obj = datetime.strptime(date_string, date_format)

    # Create a new datetime object with only month, day, and year
    date_time_only_date = datetime(year=date_time_obj.year, month=date_time_obj.month, day=date_time_obj.day)

    return date_time_only_date

In [50]:
str_to_datetime('Wed Mar 25 14:17:28 +0000 2020')

datetime.datetime(2020, 3, 25, 0, 0)

In [74]:
def extract_fields(tweet):
    tweet['created_at'] = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
    return {'created_at': tweet['created_at'], 'text': tweet['text']}

In [75]:
def order_by_datetime(tweets_data):
    if tweets_data == []:
        return []
    
    else:
        sorted_tweets = sorted(tweets_data, key=lambda x: x['created_at'], reverse=True)
        extracted_fields = [extract_fields(tweet) for tweet in sorted_tweets]
    
    return extracted_fields

In [76]:
def final_search_app(category, value, tweet_cache):
    output_lst = []
    
    cached_result = tweet_cache.get_from_cache({"category": category, "value": value})
    if cached_result:
        print("Result found in cache.")
        return cached_result["data"]
    
    if category == 'User':
        documents, documents_exist = check_doc("screen_name", value, "twitter-data-user")
        if documents_exist:
            
            user_name = documents[0]['_source']['name']
            user_screen_name = documents[0]['_source']['screen_name']
            user_profile_url = documents[0]['_source']['url']
            user_description = documents[0]['_source']['description']
            user_verification = documents[0]['_source']['verified']
            user_followers_count = documents[0]['_source']['followers_count']
            user_friends_count = documents[0]['_source']['friends_count']
            user_profile_image_url = documents[0]['_source']['profile_image_url_https']
            
            tl = convert_str_to_lst(documents[0]['_source']['tweet_ids'])
            tl_lst = []
            for l in tl:
                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                if o_exist:
                    tl_lst.append(o[0]['_source'])

            ordered_lst_tweets = order_by_datetime(tl_lst)
            
            t_lst = convert_str_to_lst(documents[0]['_source']['tweet_ids']) + convert_str_to_lst(documents[0]['_source']['retweet_ids']) + convert_str_to_lst(documents[0]['_source']['quote_ids'])
            
            for t in t_lst:
                output_dict = {}
                docs, docs_exist = check_doc("id_str", str(t), "twitter-data-tweet")
                if not docs_exist:
                    docs, docs_exist = check_doc("id_str", str(t), "twitter-data-retweet")
                    if not docs_exist:
                        docs, docs_exist = check_doc("id_str", str(t), "twitter-data-quote")
                
                if docs_exist:
                
                    output_dict['text'] = docs[0]['_source']['text']
                    output_dict['quote_count'] = docs[0]['_source']['quote_count']
                    output_dict['reply_count'] = docs[0]['_source']['reply_count']
                    output_dict['retweet_count'] = docs[0]['_source']['retweet_count']
                    output_dict['favorite_count'] = docs[0]['_source']['favorite_count']
                    output_dict['original_tweet_link'] = docs[0]['_source']['original_tweet_link']
                    output_dict['time'] = str_to_datetime(docs[0]['_source']['created_at'])
                    
                    retweet_meta_data_lst = []
                    if 'retweets' in docs[0]['_source']:
                        rt = convert_str_to_lst(docs[0]['_source']['retweets'])
                        for r in rt:
                            retweet_meta_data = {"retweet_text": "", "retweet_user": "", "retweet_time": None}
                            m, m_exist = check_doc("id_str", str(r), "twitter-data-retweet")
                            if m_exist:
                                retweet_meta_data["retweet_text"] = m[0]['_source']['text']
                                retweet_meta_data["retweet_time"] = str_to_datetime(m[0]['_source']['created_at'])
                                n, n_exist = check_doc("id_str", m[0]['_source']['retweet_user_id'], "twitter-data-user")
                                if n_exist:
                                    retweet_meta_data['retweet_user'] = n[0]['_source']['screen_name']
                            retweet_meta_data_lst.append(retweet_meta_data)
                    
                    quote_meta_data_lst = []
                    if 'quotes' in docs[0]['_source']:
                        qt = convert_str_to_lst(docs[0]['_source']['quotes'])
                        for q in qt:
                            quote_meta_data = {"quote_text": "", "quote_user": "", "quote_time": None}
                            l, l_exist = check_doc("id_str", str(q), "twitter-data-quote")
                            if l_exist:
                                quote_meta_data["quote_text"] = l[0]['_source']['text']
                                quote_meta_data["quote_time"] = str_to_datetime(l[0]['_source']['created_at'])
                                p, p_exist = check_doc("id_str", l[0]['_source']['quote_user_id'], "twitter-data-user")
                                if p_exist:
                                    quote_meta_data['retweet_user'] = p[0]['_source']['screen_name']
                            quote_meta_data_lst.append(quote_meta_data)
                    
                    output_dict['quote_metadata'] = quote_meta_data_lst
                    output_dict['retweet_metadata'] = retweet_meta_data_lst
                    output_dict['user_name'] = user_name
                    output_dict['user_screen_name'] = user_screen_name
                    output_dict['user_profile_url'] = user_profile_url
                    output_dict['user_description'] = user_description
                    output_dict['user_verification'] = user_verification
                    output_dict['user_followers_count'] = user_followers_count
                    output_dict['user_friends_count'] = user_friends_count
                    output_dict['user_profile_image_url'] = user_profile_image_url
                    output_dict['ordered_user_tweets'] = ordered_lst_tweets
                    output_lst.append(output_dict)
            
            tweet_cache.add_to_cache({"category": category, "value": value}, output_lst)
            return output_lst
    
    elif category == 'Tweet Text':
        documents_tweet, documents_exist_tweet = check_doc("text", value, "twitter-data-tweet")
        documents_retweet, documents_exist_retweet = check_doc("text", value, "twitter-data-retweet")
        documents_quote, documents_exist_quote = check_doc("text", value, "twitter-data-quote")
        
        if documents_exist_tweet or documents_exist_retweet or documents_exist_quote:
        
            documents_lst = documents_tweet + documents_retweet + documents_quote

            for t in documents_lst:
                output_dict = {}
                output_dict['text'] = t['_source']['text']
                output_dict['quote_count'] = t['_source']['quote_count']
                output_dict['reply_count'] = t['_source']['reply_count']
                output_dict['retweet_count'] = t['_source']['retweet_count']
                output_dict['favorite_count'] = t['_source']['favorite_count']
                output_dict['original_tweet_link'] = t['_source']['original_tweet_link']
                output_dict['time'] = str_to_datetime(t['_source']['created_at'])
                
                retweet_meta_data_lst = []
                if 'retweets' in t['_source']:
                    rt = convert_str_to_lst(t['_source']['retweets'])
                    for r in rt:
                        retweet_meta_data = {"retweet_text": "", "retweet_user": "", "retweet_time": None}
                        m, m_exist = check_doc("id_str", str(r), "twitter-data-retweet")
                        if m_exist:
                            retweet_meta_data["retweet_text"] = m[0]['_source']['text']
                            retweet_meta_data["retweet_time"] = str_to_datetime(m[0]['_source']['created_at'])
                            n, n_exist = check_doc("id_str", m[0]['_source']['retweet_user_id'], "twitter-data-user")
                            if n_exist:
                                retweet_meta_data['retweet_user'] = n[0]['_source']['screen_name']
                        retweet_meta_data_lst.append(retweet_meta_data)
                    
                quote_meta_data_lst = []
                if 'quotes' in t['_source']:
                    qt = convert_str_to_lst(t['_source']['quotes'])
                    for q in qt:
                        quote_meta_data = {"quote_text": "", "quote_user": "", "quote_time": None}
                        l, l_exist = check_doc("id_str", str(q), "twitter-data-quote")
                        if l_exist:
                            quote_meta_data["quote_text"] = l[0]['_source']['text']
                            quote_meta_data["quote_time"] = str_to_datetime(l[0]['_source']['created_at'])
                            p, p_exist = check_doc("id_str", l[0]['_source']['quote_user_id'], "twitter-data-user")
                            if p_exist:
                                quote_meta_data['retweet_user'] = p[0]['_source']['screen_name']
                        quote_meta_data_lst.append(quote_meta_data)
                
                output_dict['quote_metadata'] = quote_meta_data_lst
                output_dict['retweet_metadata'] = retweet_meta_data_lst
                    
                if 'tweet_user_id' in t['_source']:
                    output_dict['user_profile_url'] = t['_source']['tweet_user_profile']
                    doc, doc_exist = check_doc("id_str", t['_source']['tweet_user_id'], "twitter-data-user")
                    if doc_exist:
                        output_dict['user_name'] = doc[0]['_source']['name']
                        output_dict['user_screen_name'] = doc[0]['_source']['screen_name']
                        output_dict['user_description'] = doc[0]['_source']['description']
                        output_dict['user_verification'] = doc[0]['_source']['verified']
                        output_dict['user_followers_count'] = doc[0]['_source']['followers_count']
                        output_dict['user_friends_count'] = doc[0]['_source']['friends_count']
                        output_dict['user_profile_image_url'] = doc[0]['_source']['profile_image_url_https']
                        
                        tl = convert_str_to_lst(doc[0]['_source']['tweet_ids'])
                        tl_lst = []
                        for l in tl:
                            o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                            if o_exist:
                                tl_lst.append(o[0]['_source'])
                        
                        ordered_lst_tweets = order_by_datetime(tl_lst)
                        
                        output_dict['ordered_user_tweets'] = ordered_lst_tweets

                elif 'retweet_user_id' in t['_source']:
                    output_dict['user_profile_url'] = t['_source']['retweet_user_profile']
                    doc, doc_exist = check_doc("id_str", t['_source']['retweet_user_id'], "twitter-data-user")
                    if doc_exist:
                        output_dict['user_name'] = doc[0]['_source']['name']
                        output_dict['user_screen_name'] = doc[0]['_source']['screen_name']
                        output_dict['user_description'] = doc[0]['_source']['description']
                        output_dict['user_verification'] = doc[0]['_source']['verified']
                        output_dict['user_followers_count'] = doc[0]['_source']['followers_count']
                        output_dict['user_friends_count'] = doc[0]['_source']['friends_count']
                        output_dict['user_profile_image_url'] = doc[0]['_source']['profile_image_url_https']
                        
                        tl = convert_str_to_lst(doc[0]['_source']['tweet_ids'])
                        tl_lst = []
                        for l in tl:
                            o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                            if o_exist:
                                tl_lst.append(o[0]['_source'])
                        
                        ordered_lst_tweets = order_by_datetime(tl_lst)
                        
                        output_dict['ordered_user_tweets'] = ordered_lst_tweets

                elif 'quote_user_id' in t['_source']:
                    output_dict['user_profile_url'] = t['_source']['quote_user_profile']
                    doc, doc_exist = check_doc("id_str", t['_source']['quote_user_id'], "twitter-data-user")
                    if doc_exist:
                        output_dict['user_name'] = doc[0]['_source']['name']
                        output_dict['user_screen_name'] = doc[0]['_source']['screen_name']
                        output_dict['user_description'] = doc[0]['_source']['description']
                        output_dict['user_verification'] = doc[0]['_source']['verified']
                        output_dict['user_followers_count'] = doc[0]['_source']['followers_count']
                        output_dict['user_friends_count'] = doc[0]['_source']['friends_count']
                        output_dict['user_profile_image_url'] = doc[0]['_source']['profile_image_url_https']
                        
                        tl = convert_str_to_lst(doc[0]['_source']['tweet_ids'])
                        tl_lst = []
                        for l in tl:
                            o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                            if o_exist:
                                tl_lst.append(o[0]['_source'])
                        
                        ordered_lst_tweets = order_by_datetime(tl_lst)
                        
                        output_dict['ordered_user_tweets'] = ordered_lst_tweets
                else:
                    pass

                output_lst.append(output_dict)
        
        tweet_cache.add_to_cache({"category": category, "value": value}, output_lst)
        return output_lst
    
    elif category == 'Hashtag':
        documents, documents_exist = check_doc("text", value, "twitter-data-hashtag")
        if documents_exist:
            
            t_lst = convert_str_to_lst(documents[0]['_source']['tweet_ids']) + convert_str_to_lst(documents[0]['_source']['retweet_ids']) + convert_str_to_lst(documents[0]['_source']['quote_ids'])

            for t in t_lst:
                output_dict = {}
                docs, docs_exist = check_doc("id_str", str(t), "twitter-data-tweet")
                if not docs_exist:
                    docs, docs_exist = check_doc("id_str", str(t), "twitter-data-retweet")
                    if not docs_exist:
                        docs, docs_exist = check_doc("id_str", str(t), "twitter-data-quote")
                
                if docs_exist:
                
                    output_dict['text'] = docs[0]['_source']['text']
                    output_dict['quote_count'] = docs[0]['_source']['quote_count']
                    output_dict['reply_count'] = docs[0]['_source']['reply_count']
                    output_dict['retweet_count'] = docs[0]['_source']['retweet_count']
                    output_dict['favorite_count'] = docs[0]['_source']['favorite_count']
                    output_dict['original_tweet_link'] = docs[0]['_source']['original_tweet_link']
                    output_dict['time'] = str_to_datetime(docs[0]['_source']['created_at'])
                    
                    retweet_meta_data_lst = []
                    if 'retweets' in docs[0]['_source']:
                        rt = convert_str_to_lst(docs[0]['_source']['retweets'])
                        for r in rt:
                            retweet_meta_data = {"retweet_text": "", "retweet_user": "", "retweet_time": None}
                            m, m_exist = check_doc("id_str", str(r), "twitter-data-retweet")
                            if m_exist:
                                retweet_meta_data["retweet_text"] = m[0]['_source']['text']
                                retweet_meta_data["retweet_time"] = str_to_datetime(m[0]['_source']['created_at'])
                                n, n_exist = check_doc("id_str", m[0]['_source']['retweet_user_id'], "twitter-data-user")
                                if n_exist:
                                    retweet_meta_data['retweet_user'] = n[0]['_source']['screen_name']
                            retweet_meta_data_lst.append(retweet_meta_data)
                    
                    quote_meta_data_lst = []
                    if 'quotes' in docs[0]['_source']:
                        qt = convert_str_to_lst(docs[0]['_source']['quotes'])
                        for q in qt:
                            quote_meta_data = {"quote_text": "", "quote_user": "", "quote_time": None}
                            l, l_exist = check_doc("id_str", str(q), "twitter-data-quote")
                            if l_exist:
                                quote_meta_data["quote_text"] = l[0]['_source']['text']
                                quote_meta_data["quote_time"] = str_to_datetime(l[0]['_source']['created_at'])
                                p, p_exist = check_doc("id_str", l[0]['_source']['quote_user_id'], "twitter-data-user")
                                if p_exist:
                                    quote_meta_data['retweet_user'] = p[0]['_source']['screen_name']
                            quote_meta_data_lst.append(quote_meta_data)
                    
                    output_dict['quote_metadata'] = quote_meta_data_lst
                    output_dict['retweet_metadata'] = retweet_meta_data_lst

                    if 'tweet_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['tweet_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['tweet_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    elif 'retweet_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['retweet_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['retweet_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    elif 'quote_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['quote_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['quote_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    else:
                        pass
            
            output_lst.append(output_dict)
        
        tweet_cache.add_to_cache({"category": category, "value": value}, output_lst)
        return output_lst
    
    elif category == 'User Mention':
        
        documents, documents_exist = check_doc("screen_name", value, "twitter-data-mention")
        if documents_exist:
            
            t_lst = convert_str_to_lst(documents[0]['_source']['tweet_ids']) + convert_str_to_lst(documents[0]['_source']['retweet_ids']) + convert_str_to_lst(documents[0]['_source']['quote_ids'])

            for t in t_lst:
                output_dict = {}
                docs, docs_exist = check_doc("id_str", str(t), "twitter-data-tweet")
                if not docs_exist:
                    docs, docs_exist = check_doc("id_str", str(t), "twitter-data-retweet")
                    if not docs_exist:
                        docs, docs_exist = check_doc("id_str", str(t), "twitter-data-quote")
                        
                if docs_exist:
                    
                    output_dict['text'] = docs[0]['_source']['text']
                    output_dict['quote_count'] = docs[0]['_source']['quote_count']
                    output_dict['reply_count'] = docs[0]['_source']['reply_count']
                    output_dict['retweet_count'] = docs[0]['_source']['retweet_count']
                    output_dict['favorite_count'] = docs[0]['_source']['favorite_count']
                    output_dict['original_tweet_link'] = docs[0]['_source']['original_tweet_link']
                    output_dict['time'] = str_to_datetime(docs[0]['_source']['created_at'])
                    
                    retweet_meta_data_lst = []
                    if 'retweets' in docs[0]['_source']:
                        rt = convert_str_to_lst(docs[0]['_source']['retweets'])
                        for r in rt:
                            retweet_meta_data = {"retweet_text": "", "retweet_user": "", "retweet_time": None}
                            m, m_exist = check_doc("id_str", str(r), "twitter-data-retweet")
                            if m_exist:
                                retweet_meta_data["retweet_text"] = m[0]['_source']['text']
                                retweet_meta_data["retweet_time"] = str_to_datetime(m[0]['_source']['created_at'])
                                n, n_exist = check_doc("id_str", m[0]['_source']['retweet_user_id'], "twitter-data-user")
                                if n_exist:
                                    retweet_meta_data['retweet_user'] = n[0]['_source']['screen_name']
                            retweet_meta_data_lst.append(retweet_meta_data)
                    
                    quote_meta_data_lst = []
                    if 'quotes' in docs[0]['_source']:
                        qt = convert_str_to_lst(docs[0]['_source']['quotes'])
                        for q in qt:
                            quote_meta_data = {"quote_text": "", "quote_user": "", "quote_time": None}
                            l, l_exist = check_doc("id_str", str(q), "twitter-data-quote")
                            if l_exist:
                                quote_meta_data["quote_text"] = l[0]['_source']['text']
                                quote_meta_data["quote_time"] = str_to_datetime(l[0]['_source']['created_at'])
                                p, p_exist = check_doc("id_str", l[0]['_source']['quote_user_id'], "twitter-data-user")
                                if p_exist:
                                    quote_meta_data['retweet_user'] = p[0]['_source']['screen_name']
                            quote_meta_data_lst.append(quote_meta_data)
                    
                    output_dict['quote_metadata'] = quote_meta_data_lst
                    output_dict['retweet_metadata'] = retweet_meta_data_lst

                    if 'tweet_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['tweet_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['tweet_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    elif 'retweet_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['retweet_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['retweet_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    elif 'quote_user_id' in docs[0]['_source']:
                        output_dict['user_profile_url'] = docs[0]['_source']['quote_user_profile']
                        d, d_exist = check_doc("id_str", docs[0]['_source']['quote_user_id'], "twitter-data-user")
                        if d_exist:
                            output_dict['user_name'] = d[0]['_source']['name']
                            output_dict['user_screen_name'] = d[0]['_source']['screen_name']
                            output_dict['user_description'] = d[0]['_source']['description']
                            output_dict['user_verification'] = d[0]['_source']['verified']
                            output_dict['user_followers_count'] = d[0]['_source']['followers_count']
                            output_dict['user_friends_count'] = d[0]['_source']['friends_count']
                            output_dict['user_profile_image_url'] = d[0]['_source']['profile_image_url_https']
                            
                            tl = convert_str_to_lst(d[0]['_source']['tweet_ids'])
                            tl_lst = []
                            for l in tl:
                                o, o_exist = check_doc("id_str", str(l), "twitter-data-tweet")
                                if o_exist:
                                    tl_lst.append(o[0]['_source'])

                            ordered_lst_tweets = order_by_datetime(tl_lst)

                            output_dict['ordered_user_tweets'] = ordered_lst_tweets

                    else:
                        pass
            
            output_lst.append(output_dict)
        
        tweet_cache.add_to_cache({"category": category, "value": value}, output_lst)
        return output_lst
    
    else:
        return output_lst

In [54]:
final_search_app('Tweet Text', 'corona')

[{'text': '2k15                               2k20\nsem corona                 com corona https://t.co/4CWjhDKAsX',
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 34,
  'favorite_count': 89,
  'original_tweet_link': '',
  'time': datetime.datetime(2020, 3, 25, 0, 0),
  'quote_metadata': [],
  'retweet_metadata': [{'retweet_text': 'RT @migraineslow: 2k15                               2k20\nsem corona                 com corona https://t.co/4CWjhDKAsX',
    'retweet_user': '_fckbea_',
    'retweet_time': datetime.datetime(2020, 3, 25, 0, 0)}],
  'user_profile_url': None,
  'user_name': 'ca',
  'user_screen_name': 'migraineslow',
  'user_description': '⊬',
  'user_verification': False,
  'user_followers_count': 5030,
  'user_friends_count': 3923,
  'user_profile_image_url': 'https://pbs.twimg.com/profile_images/1248307683238658048/pCsUQe9i_normal.jpg',
  'ordered_user_tweets': [{'created_at': datetime.datetime(2020, 4, 12, 13, 54, 13, tzinfo=datetime.timezone.utc),
    'text':

In [78]:
#Imports
import json
import pandas as pd
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta
import hashlib
#from SearchApp import SearchApp

In [79]:
tweet_cache = TweetCache(max_size=30)

Refreshing cache...
Cache refreshed at: 2024-04-27 14:57:05


In [80]:
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta

# Define function to handle search
def handle_search(sender):
    start_time = datetime.now()
    keyword = search_input.value
    category = category_dropdown.value
    start_date = start_date_picker.value.strftime('%Y-%m-%d') if start_date_picker.value else None
    end_date = end_date_picker.value.strftime('%Y-%m-%d') if end_date_picker.value else None
    results = final_search_app(category, keyword, tweet_cache)
    filtered_results = filter_results_by_date(results, start_date, end_date)
    sorted_results = sort_results(filtered_results)
    end_time = datetime.now()
    search_duration = end_time - start_time
    display_results(sorted_results)
    display(HTML(f"<p>Search completed in: {search_duration}</p>"))

# Define function to filter results by date
def filter_results_by_date(results, start_date, end_date):
    if start_date and end_date:
        filtered_results = [tweet for tweet in results if start_date <= tweet.get('time', datetime.now()).strftime('%Y-%m-%d') <= end_date]
        return filtered_results
    else:
        return results

# Define function to sort results
def sort_results(results):
    if sort_dropdown.value == 'Time':
        key = lambda x: x.get('time', datetime.now())
    elif sort_dropdown.value == 'Likes':
        key = lambda x: x.get('favorite_count', 0)
    elif sort_dropdown.value == 'Retweets':
        key = lambda x: x.get('retweet_count', 0)
    elif sort_dropdown.value == 'Quotes':
        key = lambda x: x.get('quote_count', 0)
    elif sort_dropdown.value == 'Replies':
        key = lambda x: x.get('reply_count', 0)
    else:
        key = lambda x: x.get('time', datetime.now())
    sorted_results = sorted(results, key=key, reverse=sort_order_dropdown.value == 'Descending')
    return sorted_results

def display_results(results):
    output_area.clear_output()
    if not results: 
        with output_area: 
            print("No results found.") 
    else: 
        with output_area: 
            for result in results:
                display_tweet(result)


# Define function to display search results

def display_tweet(tweet):
    default_profile_image_url = "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png"
    user_profile_image_url = tweet.get('user_profile_image_url', default_profile_image_url)
    tweet_time = tweet.get('time', datetime.now()).strftime('%Y-%m-%d')
    tweet_text = tweet_text_without_retweet(tweet['text'])

    # Check if the tweet is a retweet
    is_retweet = tweet_text.startswith("RT @")
    if is_retweet:
        retweeted_user = tweet_text.split(":", 1)[0].split("RT @")[1].strip()
        retweet_prefix = f"Retweeted from @{retweeted_user}"
    else:
        retweet_prefix = ""

    # Generate a unique identifier for the tweet
    tweet_id = hashlib.md5(tweet_text.encode()).hexdigest()

    # Get retweet metadata
    retweet_metadata = tweet.get('retweet_metadata', [])
    
    # Get quote metadata
    quote_metadata = tweet.get('quote_metadata', [])

    tweet_html = f"""
        <div id="tweet_{tweet_id}" style="position: relative; border: 1px solid #ccc; border-radius: 8px; padding: 10px; margin-bottom: 10px;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="{user_profile_image_url}" style="width: 50px; height: 50px; border-radius: 50%; margin-right: 10px;" onerror="this.src='{default_profile_image_url}'; this.onerror=null;">
                <div>
                    <div><b>{tweet['user_name']}</b> @{tweet['user_screen_name']} {verified_badge(tweet['user_verification'])}</div>
                    <span style="color: #888; font-size: 14px; position: absolute; top: 5px; right: 10px;">{retweet_indicator(tweet['text'])}</span>
                    <div style="color: #888;">{tweet['user_description']}</div>
                    <div style="font-size: 12px; color: #888; float: right;">{retweet_prefix}</div>
                </div>
            </div>
            <p style="margin-bottom: 7px;">{tweet_text}</p>
            <div style="font-size: 12px; color: #888; margin-bottom: 5px;">{tweet_time}</div>
            <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
                <div>
                    <i class="fa fa-comment" style="margin-right: 5px;"></i> {tweet['reply_count']}
                    <i class="fa fa-retweet" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['retweet_count']}
                    <i class="fa fa-heart" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['favorite_count']}
                    <i class="fa fa-quote-right" style="margin-left: 10px; margin-right: 5px;"></i> {tweet['quote_count']}
                </div>
                <a href="{tweet['original_tweet_link']}" target="_blank">Original Tweet</a>
            </div>
            <div id="tweet_details_{tweet_id}" style="display: none; margin-top: 10px;">
                <b><p>Followers: </b>{tweet['user_followers_count']}</p>
                <b><p>Following: </b>{tweet['user_friends_count']}</p>
                <b><p>Website: </b>{tweet['user_profile_url']}</p>
                <h4>Recent Tweets:</h4>
                <ul>
                    {"".join(f"<li>{recent_tweet['created_at'].strftime('%Y-%m-%d')}: {recent_tweet['text']}</li>" for recent_tweet in tweet['ordered_user_tweets'])}
                </ul>
                <h4>Retweets:</h4>
                <ul>
                    {"".join(f"<li>{retweet['retweet_time'].strftime('%Y-%m-%d')}: {retweet['retweet_text']} by {retweet['retweet_user']}</li>" for retweet in retweet_metadata)}
                </ul>
                <h4>Quotes:</h4>
                <ul>
                    {"".join(f"<li>{quote['quote_time'].strftime('%Y-%m-%d')}: {quote['quote_text']} by {quote['quote_user']}</li>" for quote in quote_metadata)}
                </ul>
            </div>
            <div style="cursor: pointer; color: blue; text-decoration: underline;" onclick="toggleDetails('{tweet_id}')">
                <span id="toggle_text_{tweet_id}" style="font-weight: bold;">Show Details</span>
            </div>
        </div>
        <script>
            function toggleDetails(tweetId) {{
                var tweetDetails = document.getElementById('tweet_details_' + tweetId);
                var toggleText = document.getElementById('toggle_text_' + tweetId);
                if (tweetDetails.style.display === 'none') {{
                    tweetDetails.style.display = 'block';
                    toggleText.innerText = 'Hide Details';
                }} else {{
                    tweetDetails.style.display = 'none';
                    toggleText.innerText = 'Show Details';
                }}
            }}
        </script>
    """
    display(HTML(tweet_html))
    
# Define function to generate verified badge
def verified_badge(verified):
    if verified:
        return '<i class="fa fa-check-circle" style="color: blue; margin-left: 5px;"></i>'
    else:
        return ''

# Define function to remove "RT @username:" prefix from tweet text

def tweet_text_without_retweet(tweet_text):
    if tweet_text.startswith("RT @"):
        return tweet_text.split(":", 1)[1].strip()
    else:
        return tweet_text


def retweet_indicator(tweet_text):
    if tweet_text.startswith("RT @"):
        original_account = tweet_text.split(":")[0][4:]
        return f'Retweet from @{original_account}'
    else:
        return ''

# Define function to handle refresh
def handle_refresh(sender):
    search_input.value = ''  # Clear text input field
    category_dropdown.value = None  # Reset dropdown value
    start_date_picker.value = None  # Reset start date picker value
    end_date_picker.value = None  # Reset end date picker value
    sort_dropdown.value = 'Time'  # Reset sort dropdown value
    sort_order_dropdown.value = 'Descending'  # Reset sort order dropdown value
    search_button.disabled = True  # Disable search button
    output_area.clear_output()  # Clear output area

# Define function to update search button state
def update_search_button_state(change):
    if search_input.value and category_dropdown.value and start_date_picker.value and end_date_picker.value:
        search_button.disabled = False
    else:
        search_button.disabled = True

# Create heading
heading = widgets.HTML("<h2>Twitter Search Engine</h2>")

# Create text input widget for search query
search_input = widgets.Text(placeholder="Enter keyword", layout=widgets.Layout(width='500px'))
search_input.observe(update_search_button_state, 'value')

# Create dropdown menu for search category
category_dropdown = widgets.Dropdown(
    options=['User', 'Tweet Text', 'Hashtag', 'User Mention'],
    value=None,
    description='Category:',
    layout=widgets.Layout(width='200px')
)
category_dropdown.observe(update_search_button_state, 'value')

# Create date picker widget for start date
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    layout=widgets.Layout(width='200px')
)
start_date_picker.observe(update_search_button_state, 'value')

# Create date picker widget for end date
end_date_picker = widgets.DatePicker(
    description='End Date:',
    layout=widgets.Layout(width='200px')
)
end_date_picker.observe(update_search_button_state, 'value')

# Create dropdown menu for sorting criteria
sort_dropdown = widgets.Dropdown(
    options=['Time', 'Likes', 'Retweets', 'Quotes', 'Replies'],
    value='Time',
    description='Sort By:',
    layout=widgets.Layout(width='200px')
)

# Create dropdown menu for sort order
sort_order_dropdown = widgets.Dropdown(
    options=['Ascending', 'Descending'],
    value='Descending',
    description='Order:',
    layout=widgets.Layout(width='200px')
)

# Create search button
search_button = widgets.Button(description="Search", button_style="info", icon="search", disabled=True)
search_button.on_click(handle_search)

# Create refresh button
refresh_button = widgets.Button(description="Refresh", button_style="info", icon="refresh")
refresh_button.on_click(handle_refresh)

# Arrange widgets horizontally
form_row1 = widgets.HBox([search_input, category_dropdown, start_date_picker, end_date_picker])
form_row2 = widgets.HBox([search_button, refresh_button, sort_dropdown, sort_order_dropdown])

# Create output area to display search results
output_area = widgets.Output()

# Display the heading, form, output area, and refresh button vertically
display(widgets.VBox([heading, form_row1, form_row2, output_area]))